In [1]:
from langchain.llms import GooglePalm
import os
from dotenv import load_dotenv
load_dotenv()

google_api_key = os.environ["GOOGLE_API_KEY"]


In [7]:
google_llm = GooglePalm(google_api_key=google_api_key, temperature=0.2)

ans=google_llm('''Question:Availability stock of LEvi tshirts , prompt:You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Pay attention to use CURDATE() function to get the current date, if the question involves "today".
    
    Use the following format:
    
    Question: Question here
    SQLQuery: Query to run with no pre-amble
    
    No pre-amble.''')
 
print(ans)



Question: Availability stock of LEvi tshirts 
SQLQuery: SELECT `stock` FROM `product` WHERE `product_name` = 'Levi tshirts'


In [6]:

ans=google_llm('''Question:What is the price of 3 nike white tshirt and 1 adidas shirt? Answer:1800rs,800rs prompt:your are a inventory manager Walmart store ,you got the question from employee. 
               response professionally with the answer to employee .if multiple answers return in list format.''')

print(ans)

The price of 3 Nike white t-shirts and 1 Adidas shirt is as follows:

* Nike white t-shirt: Rs. 600 each
* Adidas shirt: Rs. 800

Therefore, the total cost is Rs. (600 x 3) + 800 = Rs. 1800.


In [3]:

db_user = os.getenv("db_user")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")
db_name = os.getenv("db_name")

from langchain.utilities import SQLDatabase

In [4]:
db_name

'Walmart_store'

In [5]:
db=SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

In [15]:
print(db.get_table_names())

['discounts', 'shoes', 't_shirts']


z:\RAG_SQL\venv\lib\site-packages\langchain\utilities\sql_database.py:278: UserWarning: This method is deprecated - please use `get_usable_table_names`.
  warnings.warn(


In [13]:
print(db.table_info)



CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER, 
	shoe_id INTEGER, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 2 and 10))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	shoe_id	pct_discount
1	1	1	5.00
2	2	2	3.50
3	3	3	4.00
*/


CREATE TABLE shoes (
	shoe_id INTEGER NOT NULL AUTO_INCREMENT, 
	model ENUM('Nike Air Force','Jordan','Adidas Ultraboost','Puma RS-X','Reebok Classic','Bata Power','Nike Zoom') NOT NULL, 
	color ENUM('Red','Blue','Black','White','Green') NOT NULL, 
	size ENUM('6','7','8','9','10') NOT NULL, 
	price_per_unit INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (shoe_id), 
	CONSTRAINT shoes_chk_1 CHECK ((`price_per_unit` between 1000 and 3500))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from shoes table:
shoe_id	model	color	size	pric

In [4]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "91"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price_per_unit * stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "22292"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price_per_unit * stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': "16725.4"} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price_per_unit * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : "17462"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "290"
     },
    {'Question': "How much sales amount will be generated if we sell all large size t-shirts today in Nike brand after discounts?",
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price_per_unit * stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike' and size='L'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer' : "290"
    }
]

to_vectorize = [" ".join(example.values()) for example in few_shots]

to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 91",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price_per_unit * stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 22292",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price_per_unit * stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 16725.4",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price_per_unit * stock_quantity) FROM

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings= HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

vs = FAISS.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [17]:

dir(vs)

# vs.similarity_search_with_score_by_vector("hello")
dir(embeddings)
# embeddings.parse_raw("hello")


{'title': 'HuggingFaceEmbeddings',
 'description': 'HuggingFace sentence_transformers embedding models.\n\nTo use, you should have the ``sentence_transformers`` python package installed.\n\nExample:\n    .. code-block:: python\n\n        from langchain.embeddings import HuggingFaceEmbeddings\n\n        model_name = "sentence-transformers/all-mpnet-base-v2"\n        model_kwargs = {\'device\': \'cpu\'}\n        encode_kwargs = {\'normalize_embeddings\': False}\n        hf = HuggingFaceEmbeddings(\n            model_name=model_name,\n            model_kwargs=model_kwargs,\n            encode_kwargs=encode_kwargs\n        )',
 'type': 'object',
 'properties': {'client': {'title': 'Client'},
  'model_name': {'title': 'Model Name',
   'default': 'sentence-transformers/all-mpnet-base-v2',
   'type': 'string'},
  'cache_folder': {'title': 'Cache Folder', 'type': 'string'},
  'model_kwargs': {'title': 'Model Kwargs', 'type': 'object'},
  'encode_kwargs': {'title': 'Encode Kwargs', 'type': 'obj